In [1]:
# 모듈 불러오기

import sys
sys.path.append("../src/model")

from spotify_recommender import Recommender

In [24]:
# fit (학습)

rec = Recommender(n_neighbors=20, metric="cosine")
rec.fit_from_csv(
    csv_path="../../dataset/processed/spotify_data_clean.csv",
    model_dir=""
)

In [25]:
# recommend (추천)
results = rec.recommend(
    by="track_name",
    query="Water Under the Bridge",
    top_k=10
)
print(results)

   rank                track_id  \
0     1  2Ej3iV0nulryy9S3UQvjSx   
1     2  6dBUzqjtbnIa1TwYbyw5CM   
2     3  6y4GYuZszeXNOXuBFsJlos   
3     4  2NiNIWrThIbzOa7zBF5J3L   
4     5  5xQGAewgneCceiarUZeBvO   
5     6  4XNrMwGx1SqP01sqkGTDmo   
6     7  3HO30hRbkOLU6ZWERayfQO   
7     8  5tdk0gDF7mmmHMcPjlY7AX   
8     9  0lL2tNyGZ84sHFQmORz803   
9    10  1atrbuO1B2ckOr652vTh6z   

                                          track_name  \
0                                   Hope That’s True   
1                                        Lovers Rock   
2                                       Kaikai Kitan   
3                          Maine Socha Ke Chura Loon   
4                                       A los amigos   
5                                     One More Night   
6                          Trouble With A Heartbreak   
7                                              Farol   
8  Mama Liked the Roses - master track with maste...   
9                                             Higher  

In [16]:
# 저장
rec.save("../models")

In [ ]:
# 불러오기
rec2 = Recommender.load("../models")

In [ ]:
# 예: 추천 결과 출력 후 장르/오디오 특성 비교
out = rec.recommend(by="track_name", query="Shape of You", top_k=10)

In [13]:
out

,rank,track_id,track_name,artist_name,distance
0,1,6OrMblzPzh6Z0pIKVeWwd3,Killers with the Cross,Powerwolf,0.018209
1,2,6PQsOjwGBqYkCoeGpSubYs,Remind You,Mimi Webb,0.018974
2,3,3L3s97R0738xGQ2VqB12qo,Drown (feat. Clinton Kane) - Alle Farben Remix,Martin Garrix,0.019345
3,4,0lQ08hbjK1umHjCYvUrmCp,Here It Is (feat. Chris Brown),Flo Rida,0.022612
4,5,0PAR95bj6egxPtdzMkgFEU,Lakes of Flame,Blue Stahli,0.022655
5,6,0SdYdws1ozBZZAGMN2EfQj,FUNKTOWN,HAVAL,0.023884
6,7,2Spzf7iZlMvUrMh8TaSpGQ,Alibi,Distrion,0.024547
7,8,3QGk8QfPKlpCCcdOUGRpMA,Fear Of Being Alone,Lennon Stella,0.025505
8,9,6XuH09i1ICYTCf1M2cJ7iS,Human Heart,Marc Benjamin,0.026678
9,10,4mUJ5LUrEYm12deebNCZSA,City Of Love,Surf Mesa,0.030022


### FAISS TEST

#### 설치항목
- conda install -c pytorch faiss-cpu 혹은 pip install faiss-cpu
- pip install scikit-learn
- pip install lightgbm
- pip install onnxmltools onnxruntime skl2onnx -> 이건 ONNX 사용할때만 하면되서 필요없을듯합니다.

In [2]:
import pandas as pd
import numpy as np
import faiss

In [3]:
df = pd.read_csv("../dataset/processed/spotify_data_clean.csv")

In [4]:
# 2. 사용할 피처 선택 (벡터화)
features = [
    "danceability", "energy", "loudness", "speechiness",
    "acousticness", "instrumentalness", "liveness", "valence", "tempo"
]
X = df[features].astype("float32").values


In [5]:
df

,idx,artist_name,track_name,track_id,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,Jason Mraz,I Won't Give Up,53QF56cjZA9RTuuMZDrSA6,68,2012,acoustic,0.483,0.303,4,-10.058,1,0.0429,0.6940,0.000000,0.1150,0.1390,133.406,240166,3
1,1,Jason Mraz,93 Million Miles,1s8tP3jP4GZcyHDsjvw218,50,2012,acoustic,0.572,0.454,3,-10.286,1,0.0258,0.4770,0.000014,0.0974,0.5150,140.182,216387,4
2,2,Joshua Hyslop,Do Not Let Me Go,7BRCa8MPiyuvr2VU3O9W0F,57,2012,acoustic,0.409,0.234,3,-13.711,1,0.0323,0.3380,0.000050,0.0895,0.1450,139.832,158960,4
3,3,Boyce Avenue,Fast Car,63wsZUhUZLlh1OsyrZq7sz,58,2012,acoustic,0.392,0.251,10,-9.845,1,0.0363,0.8070,0.000000,0.0797,0.5080,204.961,304293,4
4,4,Andrew Belle,Sky's Still Blue,6nXIYClvJAfi6ujLiKqEq8,54,2012,acoustic,0.430,0.791,6,-5.419,0,0.0302,0.0726,0.019300,0.1100,0.2170,171.864,244320,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1159743,1473391,Nicola Conte,Black Spirits,0m27F0IGHLGAWhqd6ccYst,4,2011,trip-hop,0.373,0.742,10,-6.453,0,0.0736,0.3250,0.000141,0.1590,0.5220,107.951,344013,3
1159744,1473392,Nicola Conte,Quiet Dawn,6er9p611eHEcUCU50j7D57,3,2011,trip-hop,0.516,0.675,7,-7.588,0,0.0326,0.7880,0.000129,0.1300,0.2640,119.897,285067,4
1159745,1473393,Amon Tobin,Morning Ms Candis,7jsMMqxy1tt0rH5FzYcZTQ,2,2011,trip-hop,0.491,0.440,5,-8.512,1,0.0274,0.4770,0.003130,0.0936,0.0351,100.076,214253,4
1159746,1473394,Peace Orchestra,Happy Christmas (War Is Over),77lA1InUaXztuRk2vOzD1S,0,2011,trip-hop,0.480,0.405,0,-13.343,1,0.0276,0.4310,0.000063,0.1250,0.2020,133.885,239133,3


In [6]:
class Recommender:
    def __init__(self, data: pd.DataFrame, features, nlist = 100):
        
        # 피처 벡터화
        self.df = data
        self.features = features
        self.X = data[features].astype("float32").values
        d = self.X.shape[1]

        # 3. IVF 인덱스 생성
        quantizer = faiss.IndexFlatL2(d)
        self.index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)

    def fit(self):
        print("Training FAISS IVF index...")
        self.index.train(self.X)

        # 5. 벡터 추가
        self.index.add(self.X)
        print(f"Index built. Total vectors: {self.index.ntotal}")

        # 6. 검색 편의용 매핑 (track_name, artist_name → index)
        self.track_to_idx = {name.lower(): i for i, name in enumerate(self.df["track_name"])}
        self.artist_to_idx = {}
        for i, artists in enumerate(self.df["artist_name"]):
            for artist in str(artists).split(","):
                self.artist_to_idx[artist.strip().lower()] = i
        self.id_to_idx = {tid: i for i, tid in enumerate(self.df["track_id"])}

    def recommend(self, by: str, query: str, top_k: int = 5):
        """track_id, track_name, artist_name 기준 추천"""
        query_lower = query.lower()
        if by == "track_id":
            idx = self.id_to_idx.get(query)
        elif by == "track_name":
            idx = self.track_to_idx.get(query_lower)
        elif by == "artist_name":
            idx = self.artist_to_idx.get(query_lower)
        else:
            raise ValueError(f"Unsupported search key: {by}")

        if idx is None:
            raise ValueError(f"No results found for {by}='{query}'")

        # FAISS 검색
        query_vec = self.X[idx].reshape(1, -1)
        distances, indices = self.index.search(query_vec, top_k)

        # 결과 DataFrame 생성
        results = []
        for i, r_idx in enumerate(indices[0]):
            results.append({
                "rank": i + 1,
                "track_id": self.df.loc[r_idx, "track_id"],
                "track_name": self.df.loc[r_idx, "track_name"],
                "artist_name": self.df.loc[r_idx, "artist_name"],
                "distance": float(distances[0][i]),
            })
        return pd.DataFrame(results)

In [7]:
rec = Recommender(data=df,features=features,nlist=1000)

In [8]:
rec.fit()

Training FAISS IVF index...
Index built. Total vectors: 1159748


In [9]:
recommend_item = rec.recommend(by="track_name", query="Shape of You", top_k=100)

In [10]:
recommend_item

,rank,track_id,track_name,artist_name,distance
0,1,0puxP1N7uVQuefJZOFqvZB,Shape of You,Mattias Krantz,0.000000
1,2,1jUFJrYwguTTpSddNhfmLg,Isn't She Lovely,Biréli Lagrène,0.131871
2,3,0g07YiQr2mEBW8XfZdq5FB,Change,obli,0.285162
3,4,13VAUXH3Pu2ZopLQ9EzgBQ,Heaven,Ronny Jordan,0.331956
4,5,0ldR2jGvLPFEbxzEOEzxE3,Jam On Dallas Road (feat. Dr. Mike Roma),Eddie C,0.358047
...,...,...,...,...,...
95,96,38UUQULl14Q18W5qshyM49,Bun Again,Lincoln 3dot,0.990729
96,97,25cs7bxLHEYBfsHMo1oBJv,Bat-House,Tunnel of Love,0.992017
97,98,3n0GJZW6TBneIHONltPsN5,Mwana na mpate (Remix),Alain Moloto,0.992816
98,99,7fJGmM3hmEBsgo1Aa8rMEK,Pra Virar Lobizome,Flávio José,0.993444


In [11]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [12]:
# 오디오 피처 컬럼만 선택, 필요시 리스트 수정
feature_cols = [
    'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
    'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms'
]
X = df[feature_cols]
y = df['popularity']  # 인기도 컬럼

In [13]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# LightGBM 회귀 모델 생성
model = lgb.LGBMRegressor(
    n_estimators=1000,
    num_leaves=31,
    learning_rate=0.1,
    num_threads=4,       # CPU 4코어 사용
    random_state=42
)

In [14]:
# LGBMRegressor 학습 (sklearn API)
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='rmse',
    callbacks=[
        lgb.early_stopping(stopping_rounds=100),
        lgb.log_evaluation(period=100)  # verbose 대신 사용
    ]
)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012799 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2562
[LightGBM] [Info] Number of data points in the train set: 927798, number of used features: 12
[LightGBM] [Info] Start training from score 18.383351
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 14.1887	valid_0's l2: 201.318
[200]	valid_0's rmse: 14.1231	valid_0's l2: 199.461
[300]	valid_0's rmse: 14.0834	valid_0's l2: 198.343
[400]	valid_0's rmse: 14.0598	valid_0's l2: 197.677
[500]	valid_0's rmse: 14.0449	valid_0's l2: 197.26
[600]	valid_0's rmse: 14.0364	valid_0's l2: 197.019
[700]	valid_0's rmse: 14.0289	valid_0's l2: 196.811
[800]	valid_0's rmse: 14.0227	valid_0's l2: 196.635
[900]	valid_0's rmse: 14.017	valid_0's l2: 196.477
[1000]	valid_0's rmse: 14.0127	valid_0's l2: 196.357
Did not mee

,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,1000
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [16]:
# 검증 RMSE 확인
y_pred = model.predict(X_val)
rmse = mean_squared_error(y_val, y_pred)
print(f"Validation RMSE: {rmse:.4f}")

Validation RMSE: 196.3559


In [17]:
recommend_item

,rank,track_id,track_name,artist_name,distance
0,1,0puxP1N7uVQuefJZOFqvZB,Shape of You,Mattias Krantz,0.000000
1,2,1jUFJrYwguTTpSddNhfmLg,Isn't She Lovely,Biréli Lagrène,0.131871
2,3,0g07YiQr2mEBW8XfZdq5FB,Change,obli,0.285162
3,4,13VAUXH3Pu2ZopLQ9EzgBQ,Heaven,Ronny Jordan,0.331956
4,5,0ldR2jGvLPFEbxzEOEzxE3,Jam On Dallas Road (feat. Dr. Mike Roma),Eddie C,0.358047
...,...,...,...,...,...
95,96,38UUQULl14Q18W5qshyM49,Bun Again,Lincoln 3dot,0.990729
96,97,25cs7bxLHEYBfsHMo1oBJv,Bat-House,Tunnel of Love,0.992017
97,98,3n0GJZW6TBneIHONltPsN5,Mwana na mpate (Remix),Alain Moloto,0.992816
98,99,7fJGmM3hmEBsgo1Aa8rMEK,Pra Virar Lobizome,Flávio José,0.993444


In [18]:
# df를 track_id를 인덱스로 설정
df_filtered = df.set_index('track_id').loc[recommend_item['track_id']].reset_index()

In [19]:
df_filtered.shape

(100, 20)

In [20]:
df_filtered

,track_id,idx,artist_name,track_name,popularity,year,genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0puxP1N7uVQuefJZOFqvZB,623077,Mattias Krantz,Shape of You,3,2000,acoustic,0.620,0.573,1,-7.185,0,0.0425,0.867,0.839000,0.0670,0.965,98.323,168091,4
1,1jUFJrYwguTTpSddNhfmLg,29058,Biréli Lagrène,Isn't She Lovely,20,2012,guitar,0.595,0.655,7,-7.478,1,0.0568,0.869,0.899000,0.1270,0.825,98.215,241173,3
2,0g07YiQr2mEBW8XfZdq5FB,472604,obli,Change,48,2020,soul,0.630,0.810,7,-7.402,0,0.0459,0.809,0.883000,0.1370,0.688,98.015,228040,4
3,13VAUXH3Pu2ZopLQ9EzgBQ,893207,Ronny Jordan,Heaven,31,2003,jazz,0.774,0.522,8,-7.575,1,0.0562,0.646,0.916000,0.1450,0.956,98.019,342200,4
4,0ldR2jGvLPFEbxzEOEzxE3,69515,Eddie C,Jam On Dallas Road (feat. Dr. Mike Roma),10,2013,deep-house,0.848,0.663,11,-6.977,0,0.0753,0.514,0.863000,0.1670,0.864,97.994,340453,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,38UUQULl14Q18W5qshyM49,322706,Lincoln 3dot,Bun Again,11,2018,dancehall,0.816,0.769,9,-7.539,1,0.1280,0.757,0.003010,0.0678,0.889,98.069,161633,4
96,25cs7bxLHEYBfsHMo1oBJv,944566,Tunnel of Love,Bat-House,0,2004,garage,0.525,0.726,0,-7.216,1,0.1860,0.662,0.000001,0.3560,0.636,98.305,170413,4
97,3n0GJZW6TBneIHONltPsN5,133134,Alain Moloto,Mwana na mpate (Remix),20,2014,gospel,0.590,0.825,11,-6.999,0,0.1210,0.621,0.000001,0.3860,0.892,98.450,401319,4
98,7fJGmM3hmEBsgo1Aa8rMEK,1425596,Flávio José,Pra Virar Lobizome,14,2011,forro,0.814,0.721,9,-7.100,0,0.0412,0.408,0.000000,0.1740,0.964,98.348,190880,4


In [21]:
feature_cols = [
    'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
    'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms'
]

In [22]:
X = df_filtered[feature_cols]
y = df_filtered['popularity']  # 인기도 컬럼

In [23]:
y_pred = model.predict(X)
rmse = mean_squared_error(y, y_pred)
print(f"Validation RMSE: {rmse:.4f}")

Validation RMSE: 201.4418


In [24]:
print(len(y_pred), len(recommend_item))

100 100


In [25]:
recommend_item['y_pred'] = y_pred 

In [26]:
recommend_item

,rank,track_id,track_name,artist_name,distance,y_pred
0,1,0puxP1N7uVQuefJZOFqvZB,Shape of You,Mattias Krantz,0.000000,11.024650
1,2,1jUFJrYwguTTpSddNhfmLg,Isn't She Lovely,Biréli Lagrène,0.131871,11.210709
2,3,0g07YiQr2mEBW8XfZdq5FB,Change,obli,0.285162,17.105214
3,4,13VAUXH3Pu2ZopLQ9EzgBQ,Heaven,Ronny Jordan,0.331956,14.041096
4,5,0ldR2jGvLPFEbxzEOEzxE3,Jam On Dallas Road (feat. Dr. Mike Roma),Eddie C,0.358047,16.669145
...,...,...,...,...,...,...
95,96,38UUQULl14Q18W5qshyM49,Bun Again,Lincoln 3dot,0.990729,12.794108
96,97,25cs7bxLHEYBfsHMo1oBJv,Bat-House,Tunnel of Love,0.992017,16.465442
97,98,3n0GJZW6TBneIHONltPsN5,Mwana na mpate (Remix),Alain Moloto,0.992816,11.817061
98,99,7fJGmM3hmEBsgo1Aa8rMEK,Pra Virar Lobizome,Flávio José,0.993444,13.784505


In [27]:
top10 = recommend_item.nlargest(10, 'y_pred')

In [29]:
top10

,rank,track_id,track_name,artist_name,distance,y_pred
59,60,3HVCB7SiyrHoAPNtDdqZA3,The Scientist,Eddie van der Meer,0.917534,25.472995
92,93,0XFepnAhHJu1parOgfT6DP,One Moment,Kid Francescoli,0.989588,25.233537
87,88,2EMddEnRpIvCASRpuSVQZM,Move That Mountain - Encore Version,Old Friends Quartet,0.981287,24.217966
71,72,3d79z1t2XBFJd9W0SShBHa,Calango da Lacraia,Oswaldinho do Acordeon,0.945399,23.864515
52,53,4luHDvliTCbFtz9sb2hqm0,The Winner,Emapea,0.910072,23.708622
36,37,4bHRjS3jeICc8hj349u8YV,Il fait si beau,Vincent Delerm,0.854696,23.310206
94,95,4zyE2KS55d5O2EN3Y4eavn,Heathens,Drive-By Truckers,0.990124,22.264493
89,90,0ACDb1v8pSYzlnQDFA5Z6M,Teatro da Vida,Wanderley Monteiro,0.982969,22.059424
24,25,2zeujTiAEl6o0gQPx3LiGz,Breathe,Nicholas Gunn,0.763247,21.940749
76,77,6ZUOhTs1fZem4YYsvdUEdK,Tere Liye (Instrumental),Madan Mohan,0.964129,21.928845


### ONNX 사용시

In [139]:
import lightgbm as lgb
import onnxmltools
from skl2onnx.common.data_types import FloatTensorType

In [140]:
# ONNX 변환
onnx_model = onnxmltools.convert_lightgbm(
    model,
    initial_types=[("input", FloatTensorType([None, X_train.shape[1]]))]
)

# 파일로 저장
with open("lgbm_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [141]:
import onnxruntime as ort
import numpy as np

In [155]:
for i in ort_session.get_inputs():
    print("input name:", i.name, "shape:", i.shape, "type:", i.type)

input name: input shape: [None, 12] type: tensor(float)


In [156]:
input_name

'input'

In [163]:
# ONNX Runtime 세션 생성
ort_session = ort.InferenceSession("lgbm_model.onnx")

input_name = ort_session.get_inputs()[0].name  # 첫 번째 입력 이름 가져오기

# 입력 준비 (float32 권장)
X_input = np.array(X, dtype=np.float32)
if X_input.ndim == 1:
    X_input = X_input.reshape(1, -1)

# 예측 실행
preds = ort_session.run(None, {input_name: X_input})[0]

In [164]:
preds

array([[11.024649 ],
       [11.2107115],
       [17.105215 ],
       [14.041099 ],
       [16.669146 ],
       [11.03584  ],
       [16.64258  ],
       [15.642336 ],
       [15.877936 ],
       [ 5.6580887],
       [19.300156 ],
       [ 4.088746 ],
       [10.473084 ],
       [18.689812 ],
       [15.6410885],
       [17.548199 ],
       [14.283607 ],
       [20.740845 ],
       [18.828762 ],
       [16.394669 ],
       [15.032444 ],
       [17.088024 ],
       [10.804798 ],
       [21.10416  ],
       [21.940737 ],
       [10.694089 ],
       [14.416574 ],
       [14.178171 ],
       [ 7.9605465],
       [14.454196 ],
       [16.008003 ],
       [15.20228  ],
       [15.614319 ],
       [19.826735 ],
       [13.221741 ],
       [15.871021 ],
       [23.310196 ],
       [21.678423 ],
       [15.575463 ],
       [10.593607 ],
       [11.709907 ],
       [12.12526  ],
       [-0.1543976],
       [16.838867 ],
       [ 6.0994115],
       [19.947842 ],
       [10.96133  ],
       [10.81